# National Statistical Testing – Chronic Conditions (CCHS)
This notebook performs Chi-square and Z-tests on aggregated bootstrap estimates 
to test statistical significance across demographic groups (age, sex, income, province) 
for multiple chronic conditions.


**Data Disclaimer:**  
Bootstrap estimate files derived from CCHS data are not shared in this repository due to licensing restrictions.  
The workflow and code logic are provided for transparency and reproducibility with similar datasets.


In [1]:
# ==============================================================
# National Level Statistical Testing for All Chronic Conditions
# Author: Arun Acharya
# Description:
#   Auto-detects condition sheets, performs Chi-square & Z-tests,
#   outputs one master results table for all conditions.
# ==============================================================

import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, norm

# --------------------------------------------------------------
# 1. File path (updated to placeholder for GitHub sharing)
#    Replace this with your local file path to bootstrap estimates when running.
# --------------------------------------------------------------
bootstrap_file = "PATH_TO_YOUR_BOOTSTRAP_FILE.xlsx"

# --------------------------------------------------------------
# 2. List of chronic conditions (labels truncated for sheet mapping safety)
# --------------------------------------------------------------
conditions = [
    "Sleep Apnea",
    "High Blood Pressure",
    "High Blood Cholesterol",
    "Diabetes",
    "Chronic Fatigue Syndrome",
    "Mood Disorder",
    "Anxiety Disorder",
    "Respiratory Condition",
    "Musculoskeletal Condition",
    "Cardiovascular Condition"
]

# --------------------------------------------------------------
# 3. Helper function: Chi-square test for multiple groups
# --------------------------------------------------------------
def run_chi_square(df):
    """
    Perform Chi-square test for multi-group differences.
    Expects columns: 'Estimated Prevalence (%)' and 'Weighted N'.
    """
    # Convert prevalence % to proportion
    df['Prop'] = df['Estimated Prevalence (%)'] / 100
    # Calculate weighted positives and negatives
    df['Weighted_Positive'] = df['Prop'] * df['Weighted N']
    df['Weighted_Negative'] = df['Weighted N'] - df['Weighted_Positive']
    # Prepare contingency table and run test
    contingency = df[['Weighted_Positive', 'Weighted_Negative']].to_numpy()
    chi2, p, _, _ = chi2_contingency(contingency)
    return chi2, p

# --------------------------------------------------------------
# 4. Helper function: Z-test for two groups (e.g., Male vs Female)
# --------------------------------------------------------------
def run_z_test(df, group1_name, group2_name):
    """
    Perform Z-test comparing two proportions (e.g., Male vs Female).
    Expects columns: 'Group', 'Estimated Prevalence (%)', 'Weighted N'.
    """
    g1 = df.loc[df['Group'] == group1_name]
    g2 = df.loc[df['Group'] == group2_name]
    p1, n1 = g1['Estimated Prevalence (%)'].iloc[0] / 100, g1['Weighted N'].iloc[0]
    p2, n2 = g2['Estimated Prevalence (%)'].iloc[0] / 100, g2['Weighted N'].iloc[0]
    se = np.sqrt((p1 * (1 - p1)) / n1 + (p2 * (1 - p2)) / n2)
    z = (p1 - p2) / se
    p_value = 2 * (1 - norm.cdf(abs(z)))
    return z, p_value

# --------------------------------------------------------------
# 5. Auto-detect available sheet names and build mapping dictionary
# --------------------------------------------------------------
all_sheets = pd.ExcelFile(bootstrap_file).sheet_names
# Initialize mapping for each condition and stratifier
sheet_map = {cond: {"age": None, "sex": None, "income": None, "province": None} for cond in conditions}

# Detect condition & stratifier from each sheet name
for sheet in all_sheets:
    for cond in conditions:
        if sheet.startswith(cond[:30]):  # match prefix safely (handles Excel sheet name truncation)
            lower = sheet.lower()
            if "_by_ag" in lower or "_by_age" in lower:
                sheet_map[cond]["age"] = sheet
            elif "_by_se" in lower:
                sheet_map[cond]["sex"] = sheet
            elif "_by_in" in lower:
                sheet_map[cond]["income"] = sheet
            elif "_by_pr" in lower:
                sheet_map[cond]["province"] = sheet

# Display detected mapping
print("=== Auto-generated Sheet Mapping ===")
for cond, mapping in sheet_map.items():
    print(f"{cond}: {mapping}")

# --------------------------------------------------------------
# 6. Run statistical tests for all conditions and stratifiers
# --------------------------------------------------------------
all_results = []

for cond in conditions:
    print(f"\nProcessing: {cond}")
    mapping = sheet_map[cond]
    
    # Age Group → Chi-square test
    if mapping["age"]:
        df_age = pd.read_excel(bootstrap_file, sheet_name=mapping["age"])
        chi2_age, p_age = run_chi_square(df_age)
        all_results.append([cond, "Age Group", "Chi-square", chi2_age, p_age, p_age < 0.05])
        print(f"  Age sheet used: {mapping['age']}")
    else:
        print(f"⚠ No Age sheet found for {cond}")
    
    # Sex → Z-test (Male vs Female)
    if mapping["sex"]:
        df_sex = pd.read_excel(bootstrap_file, sheet_name=mapping["sex"])
        z_sex, p_sex = run_z_test(df_sex, "Male", "Female")
        all_results.append([cond, "Sex (Male vs Female)", "Z-test", z_sex, p_sex, p_sex < 0.05])
        print(f"  Sex sheet used: {mapping['sex']}")
    else:
        print(f"⚠ No Sex sheet found for {cond}")
    
    # Income Group → Chi-square test
    if mapping["income"]:
        df_income = pd.read_excel(bootstrap_file, sheet_name=mapping["income"])
        chi2_income, p_income = run_chi_square(df_income)
        all_results.append([cond, "Income Group", "Chi-square", chi2_income, p_income, p_income < 0.05])
        print(f"  Income sheet used: {mapping['income']}")
    else:
        print(f"⚠ No Income sheet found for {cond}")
    
    # Province → Chi-square test
    if mapping["province"]:
        df_province = pd.read_excel(bootstrap_file, sheet_name=mapping["province"])
        chi2_province, p_province = run_chi_square(df_province)
        all_results.append([cond, "Province", "Chi-square", chi2_province, p_province, p_province < 0.05])
        print(f"  Province sheet used: {mapping['province']}")
    else:
        print(f"⚠ No Province sheet found for {cond}")

# --------------------------------------------------------------
# 7. Compile and save final results (export path placeholder)
# --------------------------------------------------------------
results_df = pd.DataFrame(all_results, columns=[
    "Condition", "Stratifier", "Test", "Test Statistic", "p-value", "Significant (p<0.05)"
])

output_file = "National_Significance_Results.xlsx"  # placeholder path
results_df.to_excel(output_file, index=False)

print("\n=== ALL CONDITIONS STATISTICAL TEST RESULTS ===")
print(results_df)
print(f"\nMaster results saved to: {output_file}")


=== Auto-generated Sheet Mapping ===
Sleep Apnea: {'age': 'Sleep Apnea_by_Age Group', 'sex': 'Sleep Apnea_by_Sex', 'income': 'Sleep Apnea_by_Income Group', 'province': 'Sleep Apnea_by_Province'}
High Blood Pressure: {'age': 'High Blood Pressure_by_Age Grou', 'sex': 'High Blood Pressure_by_Sex', 'income': 'High Blood Pressure_by_Income G', 'province': 'High Blood Pressure_by_Province'}
High Blood Cholesterol: {'age': 'High Blood Cholesterol_by_Age G', 'sex': 'High Blood Cholesterol_by_Sex', 'income': 'High Blood Cholesterol_by_Incom', 'province': 'High Blood Cholesterol_by_Provi'}
Diabetes: {'age': 'Diabetes_by_Age Group', 'sex': 'Diabetes_by_Sex', 'income': 'Diabetes_by_Income Group', 'province': 'Diabetes_by_Province'}
Chronic Fatigue Syndrome: {'age': 'Chronic Fatigue Syndrome_by_Age', 'sex': 'Chronic Fatigue Syndrome_by_Sex', 'income': 'Chronic Fatigue Syndrome_by_Inc', 'province': 'Chronic Fatigue Syndrome_by_Pro'}
Mood Disorder: {'age': 'Mood Disorder_by_Age Group', 'sex': 'Mood D

**Note:** Output shown here is generated from aggregated bootstrap estimates.  
The mapping confirms which sheets were detected, and the final results table summarizes 
statistical test outcomes across chronic conditions.
